# 공개 속성보다는 비공개 속성을 사용하자

Python에는 클래스 속성의 가시성`visibility`이 공개`public`와 비공개`private` 두 유형밖에 없다.

In [2]:
class MyObject(object):
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10
        
    def get_private_field(self):
        return self.__private_field

공개 속성은 어디서든 객체에 점 연산자를 사용하여 접근할 수 있다.

In [3]:
foo = MyObject()
assert foo.public_field == 5

하지만 클래스 외부에서 직접 비공개 필드에 접근하면 예외가 일어난다.

In [4]:
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

클래스 메서드도 같은 class 블록에 선언되어 있으므로 비공개 속성에 접근 할 수 있다.

In [5]:
class MyOtherObject(object):
    def __init__(self):
        self.__private_field = 71
        
    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field
    
bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

비공개 필드라는 용어에서 예상할 수 있듯이 서브클래스에서는 부모 클래스의 비공개 필드에 접근할 수 없다.

In [6]:
class MyParentObject(object):
    def __init__(self):
        self.__private_field = 71
        
class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field
    
baz = MyChildObject()
baz.get_private_field()

AttributeError: 'MyChildObject' object has no attribute '_MyChildObject__private_field'

비공개 속성의 동작은 간단하게 속성 이름을 변환하는 방식으로 구현된다. Python compiler는 `MyChildObject.get_private_field`같은 메서드에서 비공개 속성에 접근하는 코드를 발견하면 `__private_field`를 `_MyChildObject__private_field`에 접근하는 코드로 변환한다. 이 예제에서는 `__private_field`가 `MyParentObject.__init__`에만 정의되어 있으므로 비공개 속성의 실제 이름은 `_MyParentObject__private_field`가 된다. 자식 클래스에서 부모의 비공개 속성에 접근하는 동작은 단순히 변환된 속성 이름이 일치하지 않아서 실패한다.

이 체계를 이해하면 접근 권한을 확인하지 않고서도 서브클래스나 외부 클래스에서 어떤 클래스의 비공개 속성이든 쉽게 접근할 수 있다.

In [7]:
assert baz._MyParentObject__private_field == 71

객체의 속성 dictionary를 들여다보면 실제로 비공개 속성이 변환 후의 이름으로 저장되어 있음을 알 수 있다.

In [8]:
print(baz.__dict__)

{'_MyParentObject__private_field': 71}


이 외에도 속성에 접근하는 것처럼 언어 기능을 가로채는 기능이 있으면 마음만 먹으면 언제든지 객체의 내부를 조작할 수 있다.

무분별하게 객체의 내부에 접근하는 위험을 최소화 하려고 스타일 가이드에 정의된 명명 관례를 따른다. `_protected_field`처럼 앞에 밑줄 한 개를 붙인 필드는 `protected` 필드로 취급해서 클래스의 외부 사용자들이 신중하게 다뤄야 함을 의미한다.

하지만 Python을 처음 접하는 많은 프로그래머가 서브클래스나 외부에서 접근하면 안 되는 내부 API를 비공개 필드로 나타낸다.

In [9]:
class MyClass(object):
    def __init__(self, value):
        self.__value = value
        
    def get_value(self):
        return str(self.__value)
    
foo = MyClass(5)
assert foo.get_value() == '5'

이 접근 방식은 잘못되었다. 누군가는 클래스에 새 동작을 추가하거나 기존 메서드의 결함을 해결하려고 서브클래스를 만든다. __비공개 속성을 선택하면 서브클래스의 오버라이드와 확장을 다루기 어렵고 불안정하게 만들 뿐이다.__ 나중에 만들 서브 클래스에서 꼭 필요하면 여전히 비공개 필드에 접근할 수 있다.

In [10]:
class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)
    
foo = MyIntegerSubclass(5)
assert foo.get_value() == 5

나중에 클래스의 계층이 변경되면 `MyIntegerSubclass` 같은 클래스는 비공개 참조가 더는 유효하지 않게 되어 제대로 동작하지 않는다.

`MyIntegerSubclass` 클래스의 직계 부모인 `MyClass`에 `MyBaseClass`라는 또 다른 부모 클래스를 추가했다고 하자

In [20]:
class MyBaseClass(object):
    def __init__(self, value):
        self.__value = value
    
    def get_value(self):
        return str(self.__value)
    
class MyClass(MyBaseClass):
    def __init__(self, value):
        super().__init__
    
class MyIntegerSubclass(MyClass):
    def get_value(self):
        return int(self._MyClass__value)

In [21]:
foo = MyIntegerSubclass(5)
foo.get_value()

AttributeError: 'MyIntegerSubclass' object has no attribute '_MyClass__value'

일반적으로 보호 속성을 사용해서 서브클래스가 더 많은 일을 할 수 있게 하는 편이 낫다.각 보호 필드를 문서화해서 서브클래스에서 내부 API 중 어느 것을 쓸 수 있고 어느 것을 그대로 둬야 하는지 설명하자. 이렇게 하면 자신이 작성한 코들를 미래에는 안전하게 확장하는 지침이 되는 것처럼 다른 프로그래머에게도 조언이 된다.

In [22]:
class MyClass(object):
    def __init__(self, value):
        # 사용자가 객체에 전달한 값을 저장한다.
        # 문자열로 강제할 수 있는 값이어야 하며,
        # 객체에 할당하고 나면 불변으로 취급해야 한다.
        self._value = value

비공개 속성을 사용할지 진지하게 고민할 시점은 서브 클래스와 이름이 충돌할 염려가 있을 뿐이다.

이 문제는 자식 클래스가 부지불식간에 부모 클래스에서 이미 정의한 속성을 정의할 때 일언난다.

In [23]:
class ApiClass(object):
    def __init__(self):
        self._value = 5
        
    def get(self):
        return self._value
    
class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello' # 충돌
        
a = Child()
print(a.get(), 'and', a._value, 'should be different')

hello and hello should be different


주로 클래스가 공개 API의 일부일 때 문제가 된다. 서브클래스는 직접 제어할 수 없으니 문제를 고치려고 리팩토링할 수 없다.

이런 충돌은 속성 이름이 value처럼 아주 일반적일 때 일어날 확률이 특히 높다. 이런 상황이 일어날 위험을 줄이려면 부모 클래스에서 비공개 속성을 사용해서 자식 클래스와 속성 이름이 겹치지 않게 하면 된다.

In [24]:
class ApiClass(object):
    def __init__(self):
        self.__value = 5
        
    def get(self):
        return self.__value
    
class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello' # OK!
        
a = Child()
print(a.get(), 'and', a._value, 'are different')

5 and hello are different
